Ref:
    - https://github.com/bnsreenu/python_for_microscopists/blob/master/ 125_126_GAN_training_mnist.py
    - https://www.youtube.com/watch?v=Mng57Tj18pc&list=LL&index=1&t=3s&ab_channel=DigitalSreeni
Load image data ref:
    - https://machinelearningmastery.com/how-to-load-convert-and-save-images-with-the-keras-api/

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import os

1.21.5


In [15]:
# Define input image dimension
IMG_ROW, IMG_COL, CHANNEL = 24, 24, 1
IMG_SHAPE = (IMG_ROW, IMG_COL, CHANNEL)

In [33]:
# Given input of noise vector (latent).
# Generator would output fake image.
def Generator_Build():
    NOISE_SHAPE = (128, ) # 1D array
    # define generator network
    # using only dense layers here.
    # Alpha — α is a hyperparameter which controls the underlying value to which the
    # function saturates negatives network inputs.
    # Momentum — Speed up the training
    model = Sequential()
    model.add(Dense(256, input_shape=NOISE_SHAPE))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(IMG_SHAPE), activation='tanh'))
    model.add(Reshape(IMG_SHAPE))

    # model.summary()

    noise = Input(shape=NOISE_SHAPE)
    fake_img = model(noise)    #Generated image
    return Model(noise, fake_img)

In [45]:
# Define Discriminator, which outputs the likelihood of the img being real.
# Use binary classification here (real or fake)
def Discriminator_Build():
    model = Sequential()

    model.add(Flatten(input_shape=IMG_SHAPE))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))

    # model.summary()

    input_img = Input(shape=IMG_SHAPE)
    # Validity: Discriminator guess of input_img if it's real or fake.
    validity = model(input_img)
    return Model(input_img, validity)

In [19]:
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
'''Load image test part'''
l = []
test_img = load_img(r"Fonts\a1\8ea1a2f9.png", color_mode = "grayscale")
# print(type(test_img)) # PIL.Image.Image
test_img_arr = img_to_array(test_img)
# print(type(test_img_arr)) # np-array
# print(test_img_arr)
print(test_img_arr.shape)
print(test_img_arr.dtype)
l.append(test_img_arr)
print(type(l))
l = np.array(l)
print(type(l))
print(l.shape)

In [20]:
# Walk through all files in "Fonts" folder
# Fetching the image data
IMG_DATASET = []
# r=root, d=directories, f = files
for r, d, f in os.walk(r"Fonts"):
    # print("Root: ", r)
    # print("Dir: ", d)
    for file in f: # walk every file in the sub-folder(d)
        IMG_PATH = os.path.join(r, file)
        IMG = load_img(IMG_PATH, color_mode="grayscale") # load the image
        IMG = img_to_array(IMG) # convert to np-array
        if(IMG.shape[0]==24 and IMG.shape[1]==24): # ensure the size is 24*24*1
            IMG_DATASET.append(IMG)
IMG_DATASET = np.array(IMG_DATASET) # convert to np-array
# print(type(f)) # string

In [41]:
# Construct two models competing with each other.
# Define a training function, load the image dataset,
# reshape training images and set the ground truths.
def Train(epochs=500, batch_size = 64, save_interval=100):
    # load image dataset
    x_train = IMG_DATASET
    # Normalize image array to [0,1]
    x_train = x_train/255
    # Add channel dim, as shape = 24x24x1

    half_batch = int(batch_size/2)

    # Loop through (epochs) to train Discriminator by first selecting
    # a random batch of images from true dataset, generating a set of images from
    # Generator, feeding both set of images into Discriminator, and eventually setting the
    # loss parameters for both the real & fake images, as well as the combined loss. 
    for epoch in range(epochs):
        #==================== Train Discriminator ======================
        # Select a random half batch of real images
        random_pick = np.random.randint(0, x_train.shape[0], half_batch)
        real_img = x_train[random_pick]

        noise = np.random.normal(0, 1, (half_batch, 128))

        # generate a half batch of fake images
        gen_fake_img = generator.predict(noise)

        # Train Discriminator on real and fake images separately
        # losses when trained by real images
        d_loss_real = discriminator.train_on_batch(real_img, np.ones((half_batch, 1)))
        # losses when trained by fake images
        d_loss_fake = discriminator.train_on_batch(gen_fake_img, np.zeros((half_batch, 1)))
        d_avg_loss = 0.5*np.add(d_loss_real, d_loss_fake)
        #==================== Train Generator ======================
        noise = np.random.normal(0, 1, (batch_size, 128))
        # Generator wants Discriminator to label the generated samples as real(= one)
        # It's where the Generator tries to fool Discriminator that the generated images are true image. (value of 1 for y)
        valid_y = np.array([1]*batch_size)

        # Generator is part of combined where it got directly linked with the discriminator
        # Train the generator with noise as x and 1 as y. 
        # Again, 1 as the output as it is adversarial and if generator did a great
        # job of fooling the discriminator then the output would be 1 (true)
        g_loss = combined.train_on_batch(noise, valid_y)

        # In order to keep track of training process,
        # Print the progress and save the sample images output, depending on the epoch interval specified.
        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_avg_loss[0], 100*d_avg_loss[1], g_loss))
        # If at steps = save_interval,then save generated image samples
        if epoch % save_interval == 0:
            Save_img(epoch)


In [43]:
'''Save image during the progress'''
def Save_img(epoch):
    rows, cols = 5, 5 # print out rows*cols image
    noise = np.random.normal(0, 1, (rows * cols, 128))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(rows, cols)
    cnt = 0
    for i in range(rows):
        for j in range(cols):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("gen_img_fake/fake_img_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view

In [46]:
# Define optimizer
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.

# Build and compile the discriminator first. 
# Generator will be trained as part of the combined model, later. 
# Pick the loss function and the type of metric to keep track.                 
# Binary cross-entropy which it is a better for real/fake
discriminator = Discriminator_Build()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build & Compile our Discriminator, pick the loss function

# Since we are only generating (faking) images, not track any metrics.
generator = Generator_Build()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# Builds Generator and define the input noise. 
# Generator takes noise z as an input to produce its images.
noise = Input(shape=(128,))   # Random input to the generator
img = generator(noise,)

# Ensures when combining networks, only Generator is trained.
# While training generator, DO NOT adjust discriminator weights!
# Doesn't affect the descriminator training above.
discriminator.trainable = False  

# Specifies the Discriminator will take the generated images by Generator
# and take true dataset and set its output to a parameter called valid, which will indicate
# if the input is real or not.  
valid = discriminator(img,)  # Validity check on the generated image

# Combine the models and set loss function and optimizer.
# Training only the generator here.
# Goal here: Generator tries to fool the Discriminator.

# The combined model (stacked generator and discriminator) takes
# noise as input => generates images => determines validity

combined = Model(noise, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


Train(epochs=10000, batch_size=64, save_interval=500)

# Save model for future use to generate fake images
generator.save('GAN_model.h5')
                
# Epochs: the number of backward and forward propagations.
# Batch_size: the number of training samples per backward/forward propagation.
# Sample_interval: after how many epochs we call our sample_image function.

0 [D loss: 1.049012, acc.: 46.88%] [G loss: 0.978646]
1 [D loss: 0.481580, acc.: 95.31%] [G loss: 0.964249]
2 [D loss: 0.347976, acc.: 100.00%] [G loss: 0.964646]
3 [D loss: 0.319033, acc.: 95.31%] [G loss: 1.043396]
4 [D loss: 0.309942, acc.: 93.75%] [G loss: 1.128377]
5 [D loss: 0.262119, acc.: 100.00%] [G loss: 1.239538]
6 [D loss: 0.223793, acc.: 100.00%] [G loss: 1.303133]
7 [D loss: 0.209065, acc.: 98.44%] [G loss: 1.459938]
8 [D loss: 0.187478, acc.: 100.00%] [G loss: 1.598558]
9 [D loss: 0.164262, acc.: 98.44%] [G loss: 1.735209]
10 [D loss: 0.131535, acc.: 100.00%] [G loss: 1.814524]
11 [D loss: 0.115039, acc.: 100.00%] [G loss: 1.997591]
12 [D loss: 0.114016, acc.: 98.44%] [G loss: 2.131811]
13 [D loss: 0.081601, acc.: 100.00%] [G loss: 2.201300]
14 [D loss: 0.088009, acc.: 100.00%] [G loss: 2.327694]
15 [D loss: 0.072345, acc.: 100.00%] [G loss: 2.372579]
16 [D loss: 0.064807, acc.: 100.00%] [G loss: 2.443330]
17 [D loss: 0.057888, acc.: 100.00%] [G loss: 2.526985]
18 [D los

In [56]:
print(IMG_DATASET.shape)

(109100, 24, 24, 1)


In [58]:
from keras.layers import Conv2D, Conv2DTranspose

In [221]:
x = np.ones((1,32,32,1)).astype(np.float32)
x = Conv2D(64, 3, input_shape=(32,32,1), activation=LeakyReLU(alpha=0.2))(x)
x = BatchNormalization(momentum=0.8)(x)
x = Conv2D(32, 3, activation=LeakyReLU(alpha=0.2))(x)
x = BatchNormalization(momentum=0.8)(x)
x = Conv2D(32, 3, activation=LeakyReLU(alpha=0.2))(x)
x = BatchNormalization(momentum=0.8)(x)
x = Conv2D(16, 3, activation=LeakyReLU(alpha=0.2))(x)
x = BatchNormalization(momentum=0.8)(x)
x = Conv2D(1, 1, activation=LeakyReLU(alpha=0.2))(x)
x = BatchNormalization(momentum=0.8)(x)

print(x.shape)

(1, 24, 24, 1)
